<a href="https://colab.research.google.com/github/ShashankKamath/DecisionTree_ML/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np
from sklearn import svm

Saving hw2_question3.csv to hw2_question3.csv


In [3]:
data = pd.read_csv("hw2_question3.csv")
# header=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,50]
# header2=[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46]
header=[1,3,5,6,7,8,10,13,15,16,17,18,19,21,24,27,29,30,31,32,33,34,35,36,37,39,41,42,44,46,50]
header2=[2,4,9,11,12,14,20,22,23,25,26,28,38,40,43,45]

data.columns=header
for i in header2:
    data.insert(i,i,0)

data.head()

,1,3,2,5,4,6,7,8,10,9,...,39,38,41,40,42,44,43,46,45,50
0,1,1,0,1,0,1,1,-1,0,0,...,0,0,-1,0,1,1,0,1,0,-1
1,1,0,0,1,0,1,1,-1,-1,0,...,1,0,-1,0,1,0,0,-1,0,-1
2,1,0,0,1,0,1,1,-1,-1,0,...,1,0,-1,0,1,-1,0,1,0,-1
3,1,0,0,-1,0,1,1,-1,1,0,...,0,0,-1,0,1,1,0,1,0,1
4,-1,0,0,-1,0,1,-1,-1,1,0,...,1,0,-1,0,1,-1,0,-1,0,1


In [0]:
f_columns=[3,10,13,21,24,27,39,44]
for index,row in data.iterrows():
  for c_no in f_columns:
    if row[c_no]==-1:
      row[c_no-1]=1
      row[c_no]=0
      row[c_no+1]=0
    elif row[c_no]==0:
      row[c_no-1]=0
      row[c_no]=1
      row[c_no+1]=0    
    elif row[c_no]==1:
      row[c_no-1]=0
      row[c_no]=0
      row[c_no+1]=1    

In [0]:
import random
dataset=data.values.tolist()
random.shuffle(dataset)
train_set=dataset[:7369]
test_set=dataset[7370:]

In [0]:
train_array=np.array(train_set)
train_array_X = train_array[:,:-1]
train_array_Y =train_array[:,-1]
test_array=np.array(test_set)
test_array_X=test_array[:,:-1]
test_array_Y=test_array[:,-1]

In [0]:
from random import seed
from random import randrange
def cv_split(data, folds=3):
	data_split = list()
	data_copy = list(data)
	fold_size = int(len(data) / folds)
	for i in range(folds):
		fld = list()
		while len(fld) < fold_size:
			index = randrange(len(data_copy))
			fld.append(data_copy.pop(index))
		data_split.append(fld)
	return data_split

In [0]:
import time
import operator
num_of_simulations=5
# c_list=[0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]
c_list=list(np.arange(0.01,10,0.01))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
  model = svm.SVC(C=c_list[c],kernel='linear')
#   print("C: ",c_list[c])
  total_accuracy.append(0)
  total_time.append(0)
  accuracy=[]
  running_time=[]
  for i in range(0,num_of_simulations):
#     print("Simulation No: ", i)
    seed(i)
    folds = cv_split(train_set, 3)
    for x in range(len(folds)):

        CVtest_set = folds[x]
        CVtrain_set = []
        for y in range(len(folds)):
          if y != x:
            CVtrain_set.extend(folds[y])

    CVtrain_array=np.array(CVtrain_set)
    CVtrain_array_X = CVtrain_array[:,:-1]
    CVtrain_array_Y =CVtrain_array[:,-1]

    CVtest_array=np.array(CVtest_set)
    CVtest_array_X=CVtest_array[:,:-1]
    CVtest_array_Y=CVtest_array[:,-1]
    
    start_time=time.time()
    model.fit(CVtrain_array_X,CVtrain_array_Y)
    stop_time=time.time()
    running_time.append(stop_time-start_time)
    
    class_predicted=[]
    class_predicted=model.predict(CVtest_array_X)
    score=0
    for j in range(0,len(CVtest_array_Y)):
      if CVtest_array_Y[j]==class_predicted[j]:
        score+=1
    accuracy.append(score/len(class_predicted))
    
#     print("Accuracy: ", accuracy)
  total_time[c]=sum(running_time)/len(running_time)
  total_accuracy[c]=(sum(accuracy)/len(accuracy))*100

Using the Best C on Test Data

In [0]:
max_index=[i for i, x in enumerate(total_accuracy) if x == max(total_accuracy)]
max_accuracy=0
for index in range(len(max_index)):
  c_best=c_list[max_index[index]]
  model = svm.SVC(C=c_best,kernel='linear')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
print("Accuracy: ", max_accuracy)
print("C:", c)

for 5 sims
Accuracy:  94.00108577633007 
C: 0.87

for 1 sim
Accuracy:  94.10966340933767 
C: 1.6299999999999992

For Non-Linear Functions:


In [0]:
import matplotlib.pyplot as plt
plt.plot(c_list,total_accuracy)
plt.xlabel("Misclassification Cost: C [0.01:10:0.01]")
plt.ylabel("Accuracy")
plt.title("Accuracy Vs Misclassification Cost")
plt.show()

In [0]:
plt.scatter(c_list,total_time)
plt.xlabel("Misclassification Cost: C [0.01:10:0.01]")
plt.ylabel("Average Time Taken at each C (sec)")
plt.title("Average Training Time Vs Misclassification Cost")
plt.show()

Quesiton Part 2
Using Non-Linear Kernels

In [0]:
import time
num_of_simulations=1
c_list=list(np.arange(0.01,100,0.1))
gamma_list=list(np.arange(0.0001,1,0.1))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(gamma_list)):
      model = svm.SVC(C=c_list[c],gamma=gamma_list[g],kernel='rbf')
    #   print("C: ",c_list[c])

      accuracy=[]
      running_time=[]
      for i in range(0,num_of_simulations):
    #     print("Simulation No: ", i)
        seed(i)
        folds = cv_split(train_set, 3)
        for x in range(len(folds)):

            CVtest_set = folds[x]
            CVtrain_set = []
            for y in range(len(folds)):
              if y != x:
                CVtrain_set.extend(folds[y])

        CVtrain_array=np.array(CVtrain_set)
        CVtrain_array_X = CVtrain_array[:,:-1]
        CVtrain_array_Y =CVtrain_array[:,-1]

        CVtest_array=np.array(CVtest_set)
        CVtest_array_X=CVtest_array[:,:-1]
        CVtest_array_Y=CVtest_array[:,-1]

        start_time=time.time()
        model.fit(CVtrain_array_X,CVtrain_array_Y)
        stop_time=time.time()
        running_time.append(stop_time-start_time)

        class_predicted=[]
        class_predicted=model.predict(CVtest_array_X)
        score=0
        for j in range(0,len(CVtest_array_Y)):
          if CVtest_array_Y[j]==class_predicted[j]:
            score+=1
        accuracy.append(score/len(class_predicted))

    #     print("Accuracy: ", accuracy)
      total_time[c].append(sum(running_time)/len(running_time))
      total_accuracy[c].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
max_index = np.where(total_accuracy == np.amax(total_accuracy))

max_accuracy=0
for index in range(len(max_index[0])):
  c_best=c_list[max_index[0][index]]
  g_best=gamma_list[max_index[1][index]]
  model = svm.SVC(C=c_best,gamma=g_best,kernel='rbf')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
    g=g_best
print("Accuracy: ", max_accuracy)
print("C:", c)
print("Gamma: ",g)

Accuracy:  97.04125950054289
C: 1.2100000000000002
Gamma: 0.10010000000000001

In [0]:
import time
num_of_simulations=1
c_list=list(np.arange(0.01,100,0.1))
degree=[1,2,3,4,5]
gamma_list=list(np.arange(0.0001,1,0.1))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(degree)):
      model = svm.SVC(C=c_list[c],degree=degree[g],gamma=0.1,kernel='poly')
    #   print("C: ",c_list[c])

      accuracy=[]
      running_time=[]
      for i in range(0,num_of_simulations):
    #     print("Simulation No: ", i)
        seed(i)
        folds = cv_split(train_set, 3)
        for x in range(len(folds)):

            CVtest_set = folds[x]
            CVtrain_set = []
            for y in range(len(folds)):
              if y != x:
                CVtrain_set.extend(folds[y])

        CVtrain_array=np.array(CVtrain_set)
        CVtrain_array_X = CVtrain_array[:,:-1]
        CVtrain_array_Y =CVtrain_array[:,-1]

        CVtest_array=np.array(CVtest_set)
        CVtest_array_X=CVtest_array[:,:-1]
        CVtest_array_Y=CVtest_array[:,-1]

        start_time=time.time()
        model.fit(CVtrain_array_X,CVtrain_array_Y)
        stop_time=time.time()
        running_time.append(stop_time-start_time)

        class_predicted=[]
        class_predicted=model.predict(CVtest_array_X)
        score=0
        for j in range(0,len(CVtest_array_Y)):
          if CVtest_array_Y[j]==class_predicted[j]:
            score+=1
        accuracy.append(score/len(class_predicted))

    #     print("Accuracy: ", accuracy)
      total_time[c].append(sum(running_time)/len(running_time))
      total_accuracy[c].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
max_index = np.where(total_accuracy == np.amax(total_accuracy))

max_accuracy=0
for index in range(len(max_index[0])):
  c_best=c_list[max_index[0][index]]
  d_best=degree[max_index[1][index]]
  model = svm.SVC(C=c_best,degree=d_best,gamma=0.1,kernel='poly')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
    d=d_best
print("Accuracy: ", max_accuracy)
print("C:", c)
print("Degree: ",d)

Poly
Accuracy:  96.66123778501628
C: 0.01
G=0.1
Degree:  5


Accuracy:  95.98262757871878
C: 0.01
Gamma: 1
Degree:  2

{'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
Accuracy: 96.30836047774159

In [9]:
import time
num_of_simulations=1
c_list=[1.27]
gamma_list=[0.1]
##GAMMA after 1 waste
#c_list=list(np.arange(0.01,100,0.1))
#
#gamma_list=list(np.arange(0.0001,1,0.1))
#c_list=[0.1,1]
#gamma_list=[0.1]
degree=[5]
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(gamma_list)):
        total_accuracy[c].append([])
        total_time[c].append([])
        for d in range(len(degree)):
          model = svm.SVC(C=c_list[c],degree=degree[d],gamma=gamma_list[g],kernel='poly')
        #   print("C: ",c_list[c])

          accuracy=[]
          running_time=[]
          for i in range(0,num_of_simulations):
        #     print("Simulation No: ", i)
            seed(i)
            folds = cv_split(train_set, 3)
            for x in range(len(folds)):

                CVtest_set = folds[x]
                CVtrain_set = []
                for y in range(len(folds)):
                  if y != x:
                    CVtrain_set.extend(folds[y])

            CVtrain_array=np.array(CVtrain_set)
            CVtrain_array_X = CVtrain_array[:,:-1]
            CVtrain_array_Y =CVtrain_array[:,-1]

            CVtest_array=np.array(CVtest_set)
            CVtest_array_X=CVtest_array[:,:-1]
            CVtest_array_Y=CVtest_array[:,-1]

            start_time=time.time()
            model.fit(CVtrain_array_X,CVtrain_array_Y)
            stop_time=time.time()
            running_time.append(stop_time-start_time)

            class_predicted=[]
            class_predicted=model.predict(CVtest_array_X)
            score=0
            for j in range(0,len(CVtest_array_Y)):
              if CVtest_array_Y[j]==class_predicted[j]:
                score+=1
            accuracy.append(score/len(class_predicted))

        #     print("Accuracy: ", accuracy)
          total_time[c][g].append(sum(running_time)/len(running_time))
          total_accuracy[c][g].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
ind = np.unravel_index(np.argmax(total_accuracy, axis=None), total_accuracy.shape)
model = svm.SVC(C=c_list[ind[0]],degree=degree[ind[2]],gamma=gamma_list[ind[1]],kernel='poly')
model.fit(train_array_X,train_array_Y)
y_predict=model.predict(test_array_X)
score=0
for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
        score+=1
accuracy=(score/len(y_predict))*100
print("Accuracy: ", accuracy)
print("C:", c_list[ind[0]])
print("Gamma:",gamma_list[ind[1]])
print("Degree: ",degree[ind[2]])

Accuracy:  96.22692725298589
C: 1.27
Gamma: 0.1
Degree:  5


In [0]:
total_time


In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
Axes3D.scatter(range(len(total_)), ys, zs=0,